# Generating images with DM-Font model from a reference style
In this example we'll generate images with trained DM-Font model from a reference style.
If you want to generate multiple styles, please check using `inference.py` instead of using this example file (because it is much simpler to load the referece styles).

### 1. Loading packages
* First, load the packages used in this code.
* All of the packages are avilable in `pip`.

In [3]:
import json
from pathlib import Path
from PIL import Image

import torch
from sconf import Config
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

* These modules are defined in this repository.

In [4]:
from base.dataset import read_font, render
from base.utils import save_tensor_to_image, load_reference
from DM.models import Generator
from inference import infer_DM

### 2. Build model
* Build and load the trained model.
* `weight_path` : 
    * The location of the trained model weight.
* `decomposition` : 
    * The location of the pre-defined decomposition rule file.
* `n_heads` :
    * The number of heads. 3 for the Korean script.
* `n_comps` :
    * The number of total components. 68 for the Korean script.

In [5]:
###############################################################
weight_path = "weights/kor/dmfont.pth"  # path to weight to infer
decomposition = "data/kor/decomposition_DM.json"
n_heads = 3
n_comps = 68
###############################################################

# building and loading the model (not recommended to modify)
cfg = Config("cfgs/DM/default.yaml")
decomposition = json.load(open(decomposition))

gen = Generator(n_heads=n_heads, n_comps=n_comps).cuda().eval()
weight = torch.load(weight_path)
gen.load_state_dict(weight["generator_ema"])

/media/mrjaehong/DATA/python/fewshot-font-generation/env/lib/python3.7/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


FileNotFoundError: [Errno 2] No such file or directory: 'weights/kor/dmfont.pth'

### 3. Load reference images.
* `ref_path`: 
    * The path of reference font or images.
    * If you are using a ttf file, set this to the location of the ttf file.
    * If you want to use rendered images, set this to the path to the directory which contains the reference images.
* `ref_chars`:
    * The characters of reference images.
    * If this is `None`, all the available images will be loaded.
* `extension`:
    * If you are using ttf files, set this to "ttf".
    * If you are using image files, set this to their extension(png, jpg, etc..).

In [ ]:
###############################################################
ref_path = "data_example"
extension = "png"
ref_chars = None
## Comment upper lines and uncomment lower lines to test with ttf files.
# extension = "ttf"
# ref_chars = "값같곬곶깎넋늪닫닭닻됩뗌략몃밟볘뺐뽈솩쐐앉않얘얾엌옳읊죡쮜춰츄퀭틔핀핥훟"
###############################################################

ref_dict, load_img = load_reference(ref_path, extension, ref_chars)

### 4. Generate and save the images.
* `gen_chars`: The characters to generate.
* `save_dir`: Path to save the generated images.
* `batch_size`: The number of images inferred at once.

In [ ]:
###############################################################
gen_chars = "좋은하루되세요"  # characters to generate
save_dir = "./result/dm"
batch_size = 16
###############################################################

infer_DM(gen, save_dir, gen_chars, ref_dict, load_img, decomposition, batch_size)